In [1]:
import boto3
import json
from datetime import datetime
import pytz


region = 'us-east-1'
client = boto3.client('stepfunctions', region_name = region)

In [2]:
# failed_execution_name = 'ce001005-a92a-44c2-abd6-a968d222f78e_152106'
# state_machine_name = 'prod-incoming-leads-business-process'

# Supply Failed State Machine Name
state_machine_name = 'prod-incoming-leads-business-process'
state_machine_arn = f"arn:aws:states:us-east-1:133240237239:stateMachine:{state_machine_name}"

In [3]:
# Get list of executions 

response = client.list_executions(
    stateMachineArn=state_machine_arn,
    statusFilter='FAILED',
    maxResults=1000
)

# print(response)
if response.get("nextToken", None):
    print(f"More executions exist, loop through for pagination using next token {response.get('nextToken')}")
print(f"total executions = {len(response['executions'])}")


More executions exist, loop through for pagination using next token AAAAKgAAAAIAAAAAAAAAAUZpPOs20/20l3Kv73NRBbz13XL18WaMbLNR7BMa6cZL2+Je4y2t4V9LP7tm976e4kB2BJt0Q4ifDDwERwhON3WJKWEAU1teZz2zdK0hp4z24sHyRLFfmmk80wyXK4XvdPlW3gfVGb2524QrmBZ6Ty8R5p8iVoypmrfuUNr/Ghnlx9iY9HTxiSkyw1eKRflgfhFoIeNIMgRiLh2EWWfb2rUw+VNRKqfHYs9Et8BCs3KpoF1lvaMpVRMjTsokMtcdx6bhZi6erEDOzvGa1tvUUcFq5rz21ycoFqOWvryYk7hMcsR2ddg1SHxtIlZJEKZd7PZY56iIqMWIu46z7pZFoqvjiNcY66s54l5Q90vn+SkmrkvpVxNOfKb4301UA74JXhz2VdY89BiWuk82sMliIkn4bmV0okvQn7G39Lx4s84F4Y/d70O/eg2YgZtVZnX+wCO+v047FdMey8WFl/LB8wW32X7gqi8Di5DBTJiOE3bC
total executions = 1000


In [ ]:

response_1000_2000 = client.list_executions(
    stateMachineArn=state_machine_arn,
    statusFilter='FAILED',
    maxResults=1000,
    nextToken="AAAAKgAAAAIAAAAAAAAAAbjYh3FIkk5xe+Gw90qIqn86KkdJdO/uqtuiP6qp9Zc/jB46hvH/fRTVk3r1c67dgYrODj4UetPq6L7I+jTGdmp7gPmh2UeiLbXfI1mzyliXnLDXKzBFVNH3HsLq3ml1kwlvb/eZ+k6S5Exd5xxKuFGCX1ZY1A83eXzhVLnA/LRAxmVVGNZSPR1u7I5y2xOQkeF6b4/7LTLR1UUKxE5uCAiyToH0PWk6Yc8ZqpYBG4bnJNjIbHNshCneKAuFH23i1RdtQlFhRKTST8u7etETTEXw5++/wHanZ+MeX1QLPBCzQrnAfYUlf64gKo6m92DIabDCbLu4jNUyMB1hKP8Eq2ayK3fmV+64SaTZLIxJrK4o8j0K6kdHuttkUWunPGFFvaNusyuIqDnfyTDMQOkIj/1f3BrtB2LHnOIXPIA6EwbNF6iBFEUjG/PUVIPzizq8NoNH+0kzU4GclVyaKvuXtGH67c8NJh8YDIWJmSZozAE7"
)
# print(response)
if response_1000_2000.get("nextToken", None):
    print(f"More executions exist, loop through for pagination using next token {response_1000_2000.get('nextToken')}")
print(f"total executions = {len(response_1000_2000['executions'])}")



In [10]:

response_2000_3000 = client.list_executions(
    stateMachineArn=state_machine_arn,
    statusFilter='FAILED',
    maxResults=1000,
    nextToken="AAAAKgAAAAIAAAAAAAAAATI+FZXyvtLZ9L3OcDf1gl/xlpHRxK7RBc/Vx1q1NJrKKDshjfjaWRZEZ3lgLcCIIb5QcmCQdp55B5TOSpv6cl8eE0VVonZYEh8/5tTianaL54tMdTG/Ius7LCaSD678sem5RXvkqOr3dA5MP02kIVl4iq6JNszMiD/wP5Ta0nYfFFMYBTVe0+8eE3+2BD/5NZuEwcBwxX4uTD7fGy1yoItcnmghTicqkL357ry/4mACNaUaAV7d4b++um7ZXR3nRrgz6c6D7vy9zG9hi1nFbG/htAFfLfNF4BKKRZ2cRBks7bVrv1k7d0TRnFoPVprpB3pFyyzz2TZhfH5LJCjpTwizzA8m0WEocQ/t20yu4E7CBZhz6BqwHTHt/2+F66cAlHrErAODigbZsADqOEFufOOBaTMTG4Bypu2fqkc2u11edouGW/Ah3omRpaBaEro8lLAP9pCnGcz6vTJK5I1JI02TxcNyn/zuD9xnvXAQl1tx"
)

# print(response)
if response_2000_3000.get("nextToken", None):
    print(f"More executions exist, loop through for pagination using next token {response_2000_3000.get('nextToken')}")
print(f"total executions = {len(response_2000_3000['executions'])}")



total executions = 620


In [7]:
timezone = pytz.timezone('America/New_York')

min_start_date_time_str = '2021-06-09 00:00:00.000000'
min_start_date_time_obj = timezone.localize(datetime.strptime(min_start_date_time_str, '%Y-%m-%d %H:%M:%S.%f'))
max_start_date_time_str = '2021-06-07 23:20:00.000000'
max_start_date_time_obj = timezone.localize(datetime.strptime(max_start_date_time_str, '%Y-%m-%d %H:%M:%S.%f'))

failed_execution_arns = []

for execution in response['executions']:
    if min_start_date_time_obj <= execution['startDate']:
        failed_execution_arns.append(execution['executionArn'])

        


In [ ]:
        
for execution in response_1000_2000['executions']:
    if min_start_date_time_obj <= execution['startDate']:
        failed_execution_arns.append(execution['executionArn'])
        
for execution in response_2000_3000['executions']:
    if min_start_date_time_obj <= execution['startDate']:
        failed_execution_arns.append(execution['executionArn'])

In [8]:
print(failed_execution_arns)

['arn:aws:states:us-east-1:133240237239:execution:prod-incoming-leads-business-process:7f977963-8ae5-4cfb-95f3-0cc8fba0b942_041117', 'arn:aws:states:us-east-1:133240237239:execution:prod-incoming-leads-business-process:fc2d5031-c243-48ad-90ff-99cfe66cb5d0_041118', 'arn:aws:states:us-east-1:133240237239:execution:prod-incoming-leads-business-process:104d2dce-0174-4a89-9a9b-245749b14f88_041111', 'arn:aws:states:us-east-1:133240237239:execution:prod-incoming-leads-business-process:5a321ad5-1403-4081-9c0c-cd10f6f68bf7_041018', 'arn:aws:states:us-east-1:133240237239:execution:prod-incoming-leads-business-process:0ff480a0-502f-43bd-96c1-0994dcf9e45d_041013']


In [9]:
# failed_execution_arn = f'arn:aws:states:us-east-1:133240237239:execution:{state_machine_name}:{failed_execution_name}'
with open("failed_executions_remaining.txt", "w") as file:
    for fea in failed_execution_arns:
        file.write(fea + "\n")

In [20]:
def parseFailureHistory(failedExecutionArn):
    '''
    Parses the execution history of a failed state machine to get the name of failed state and
    the input to the failed state
    Input failedExecutionArn - a string containing the execution Arn of a failed state machine
    Output - a list with two elements: [name of failed state, input to failed state]
    '''

    failedEvents = list()
    failedAtParallelState = False

    try:
        # Get the execution history
        response = client.get_execution_history(
            executionArn=failedExecutionArn,
            reverseOrder=True
        )
        next_token = response.get('nextToken')
        failedEvents.extend(response['events'])
    except Exception as ex:
        raise ex

    while next_token is not None:
        try:
            # Get the execution history
            response = client.get_execution_history(
                executionArn=failedExecutionArn,
                reverseOrder=True,
                nextToken=next_token
            )
            next_token = response.get('nextToken')
            failedEvents.extend(response['events'])
        except Exception as ex:
            raise ex

    # Confirm that the execution actually failed, raise exception if it didn't fail
    try:
        failedEvents[0]['executionFailedEventDetails']
    except:
        raise ('Execution did not fail')
    '''
    If we have a 'States.Runtime' error (for example if a task state in our state 
    machine attempts to execute a lambda function in a different region than the 
    state machine, get the id of the failed state, use id of the failed state to
    determine failed state name and input
    '''
    if failedEvents[0]['executionFailedEventDetails']['error'] == 'States.Runtime':
        failedId = int(filter(str.isdigit, str(failedEvents[0]['executionFailedEventDetails']['cause'].split()[13])))
        failedState = failedEvents[-1 * failedId]['stateEnteredEventDetails']['name']
        failedInput = failedEvents[-1 * failedId]['stateEnteredEventDetails']['input']
        return (failedState, failedInput)
    '''
    We need to loop through the execution history, tracing back the executed steps
    The first state we encounter will be the failed state
    If we failed on a parallel state, we need the name of the parallel state rather than the 
    name of a state within a parallel state it failed on. This is because we can only attach
    the goToState to the parallel state, but not a sub-state within the parallel state.
    This loop starts with the id of the latest event and uses the previous event id's to trace
    back the execution to the beginning (id 0). However, it will return as soon it finds the name
    of the failed state 
    '''
    currentEventId = failedEvents[0]['id']
    while currentEventId != 0:
        # multiply event id by -1 for indexing because we're looking at the reversed history
        currentEvent = failedEvents[-1 * currentEventId]
        '''
        We can determine if the failed state was a parallel state because it an event
        with 'type'='ParallelStateFailed' will appear in the execution history before
        the name of the failed state
        '''
        if currentEvent['type'] == 'ParallelStateFailed':
            failedAtParallelState = True
        '''
        If the failed state is not a parallel state, then the name of failed state to return
        will be the name of the state in the first 'TaskStateEntered' event type we run into 
        when tracing back the execution history
        '''
        if currentEvent['type'] == 'TaskStateEntered' and failedAtParallelState == False:
            failedState = currentEvent['stateEnteredEventDetails']['name']
            failedInput = currentEvent['stateEnteredEventDetails']['input']
            return (failedState, failedInput)
        '''
        If the failed state was a paralell state, then we need to trace execution back to 
        the first event with 'type'='ParallelStateEntered', and return the name of the state
        '''
        if currentEvent['type'] == 'ParallelStateEntered' and failedAtParallelState:
            failedState = currentEvent['stateEnteredEventDetails']['name']
            failedInput = currentEvent['stateEnteredEventDetails']['input']
            return (failedState, failedInput)
        # Update the id for the next execution of the loop
        currentEventId = currentEvent['previousEventId']

In [37]:
key_to_search = "Phone"
value_to_search = "606-877-5842"
list_to_rerun = []
for failed_execution_arn in failed_execution_arns:
    failedState, failedInput = parseFailureHistory(failed_execution_arn)
    failedInputObj = json.loads(failedInput)
    if failedInputObj.get(key_to_search, None) and value_to_search in failedInputObj.get(key_to_search):
        print(failed_execution_arn, failedState, failedInputObj[key_to_search])
        list_to_rerun.append(failed_execution_arn)
        
print(list_to_rerun)
    

arn:aws:states:us-east-1:133240237239:execution:prod-incoming-lead-form-process:35bfa958-ef27-44bd-b7ed-25fc26b81304_205339 postLeadFormDataToLeadspedia 606-877-5842
arn:aws:states:us-east-1:133240237239:execution:prod-incoming-lead-form-process:d1d33e08-67d8-4c9d-9926-06e190551963_205339 postLeadFormDataToLeadspedia 606-877-5842
arn:aws:states:us-east-1:133240237239:execution:prod-incoming-lead-form-process:cccf3ada-2c9a-4599-aaa2-001c9ca10800_205317 postLeadFormDataToLeadspedia 606-877-5842
arn:aws:states:us-east-1:133240237239:execution:prod-incoming-lead-form-process:1437f61c-5fbc-4340-bf48-2fc157d15770_205317 postLeadFormDataToLeadspedia 606-877-5842
['arn:aws:states:us-east-1:133240237239:execution:prod-incoming-lead-form-process:35bfa958-ef27-44bd-b7ed-25fc26b81304_205339', 'arn:aws:states:us-east-1:133240237239:execution:prod-incoming-lead-form-process:d1d33e08-67d8-4c9d-9926-06e190551963_205339', 'arn:aws:states:us-east-1:133240237239:execution:prod-incoming-lead-form-process:

In [ ]:
failed_response

In [15]:
failedEvents = [] 

In [55]:
failedEvents.extend(failed_response['events'])

In [56]:
failedEvents[0]['executionFailedEventDetails']

{'error': 'States.Timeout'}

In [21]:
currentEventId = failedEvents[0]['id']
while currentEventId != 0:
    # multiply event id by -1 for indexing because we're looking at the reversed history
    currentEvent = failedEvents[-1 * currentEventId]
    '''
    We can determine if the failed state was a parallel state because it an event
    with 'type'='ParallelStateFailed' will appear in the execution history before
    the name of the failed state
    '''
    if currentEvent['type'] == 'ParallelStateFailed':
        failedAtParallelState = True
    '''
    If the failed state is not a parallel state, then the name of failed state to return
    will be the name of the state in the first 'TaskStateEntered' event type we run into 
    when tracing back the execution history
    '''
    if currentEvent['type'] == 'TaskStateEntered' and failedAtParallelState == False:
        failedState = currentEvent['stateEnteredEventDetails']['name']
        failedInput = currentEvent['stateEnteredEventDetails']['input']
#         print (failedState, failedInput)
        print (failedState)
    '''
    If the failed state was a paralell state, then we need to trace execution back to 
    the first event with 'type'='ParallelStateEntered', and return the name of the state
    '''
    if currentEvent['type'] == 'ParallelStateEntered' and failedAtParallelState:
        failedState = failedState = currentEvent['stateEnteredEventDetails']['name']
        failedInput = currentEvent['stateEnteredEventDetails']['input']
#         print (failedState, failedInput)
        print (failedState)
    # Update the id for the next execution of the loop
    currentEventId = currentEvent['previousEventId']

FinalFanout


In [ ]:
failedStateName = 'FinalFanout'
failedInput

In [22]:
def smArnFromExecutionArn(arn):
    '''
    Get the State Machine Arn from the execution Arn
    Input: Execution Arn of a state machine
    Output: Arn of the state machine
    '''
    smArn = arn.split(':')[:-1]
    smArn[5] = 'stateMachine'
    return ':'.join(smArn)

In [60]:
smArn = smArnFromExecutionArn(failed_execution_arn)
smArn

'arn:aws:states:us-east-1:133240237239:stateMachine:prod-incoming-leads-business-process'

In [90]:
response = client.describe_state_machine(
            stateMachineArn=smArn
        )

In [91]:
response

{'stateMachineArn': 'arn:aws:states:us-east-1:133240237239:stateMachine:prod-incoming-leads-business-process',
 'name': 'prod-incoming-leads-business-process',
 'status': 'ACTIVE',
 'definition': '{\n  "Comment": "Business process flow for leads",\n  "StartAt": "insertNonLeadFormDataInToDynamo",\n  "States": {\n    "insertNonLeadFormDataInToDynamo": {\n      "Type": "Task",\n      "Comment": "Insert all non lead form data into Dynamo DB",\n      "Resource": "arn:aws:lambda:us-east-1:133240237239:function:integriant-leadintake-api-prod-allLeadDynamoInsert",\n      "TimeoutSeconds": 60,\n      "Retry": [\n        {\n          "ErrorEquals": [\n            "HandledError",\n            "States.TaskFailed",\n            "Lambda.Unknown"\n          ],\n          "IntervalSeconds": 10,\n          "MaxAttempts": 2,\n          "BackoffRate": 2\n        },\n        {\n          "ErrorEquals": [\n            "States.ALL"\n          ],\n          "IntervalSeconds": 5,\n          "MaxAttempts": 2,\

In [92]:
from datetime import datetime
now_str = datetime.now().strftime("%Y%m%d")

In [93]:
import json

roleArn = response['roleArn']
stateMachine = json.loads(response['definition'])
# Create a name for the new state machine
newName = failed_execution_name + '-FR-' + now_str
# Get the StartAt state for the original state machine, because we will point the 'GoToState' to this state
originalStartAt = stateMachine['StartAt']
'''
Create the GoToState with the variable $.resuming
If new state machine is executed with $.resuming = True, then the state machine will skip to the failed state
Otherwise, it will execute the state machine from the original start state
'''
goToState = {'Type': 'Choice',
             'Choices': [{'Variable': '$.resuming', 'BooleanEquals': False, 'Next': originalStartAt}],
             'Default': failedStateName}
# Add GoToState to the set of states in the new state machine
stateMachine['States']['GoToState'] = goToState
# Add StartAt
stateMachine['StartAt'] = 'GoToState'
# Create new state machine
try:
    response = client.create_state_machine(
        name=newName,
        definition=json.dumps(stateMachine),
        roleArn=roleArn
    )
except:
    raise ('Failed to create new state machine with GoToState')
print (response)

{'stateMachineArn': 'arn:aws:states:us-east-1:133240237239:stateMachine:ce001005-a92a-44c2-abd6-a968d222f78e_152106-FR-20210526', 'creationDate': datetime.datetime(2021, 5, 26, 10, 55, 37, 877000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '7d32ad0e-e768-467e-bbe4-1383dcc1fc05', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7d32ad0e-e768-467e-bbe4-1383dcc1fc05', 'content-type': 'application/x-amz-json-1.0', 'content-length': '160'}, 'RetryAttempts': 0}}


In [40]:
failed_execution_name

'ce001005-a92a-44c2-abd6-a968d222f78e_152106'

In [106]:
new_statemachine_arn = 'arn:aws:states:us-east-1:133240237239:stateMachine:ce001005-a92a-44c2-abd6-a968d222f78e_152106-FR-20210526'

In [76]:
failed_input = json.loads(failedInput)

In [ ]:
json.dumps(failed_input)

In [104]:
response["stateMachineArn"]

KeyError: 'stateMachineArn'

In [105]:
new_sm_name = stateMachineArn=response["stateMachineArn"].split(":")[-1]

KeyError: 'stateMachineArn'

In [102]:
failed_input["resuming"] = True

In [100]:
new_sm_arn

'ce001005-a92a-44c2-abd6-a968d222f78e_152106-FR-20210526'

In [107]:
response = client.start_execution(
    stateMachineArn=new_statemachine_arn,
    name=new_sm_arn + "-" +  datetime.now().strftime("%H%M%S"),
    input=json.dumps(failed_input)
)

In [109]:
try:
    response = client.delete_state_machine(
        stateMachineArn=new_statemachine_arn
    )
except:
    raise ('Failed to create new state machine with GoToState')
print (response)

{'ResponseMetadata': {'RequestId': '3095eedd-ae83-4098-a8c3-d0c183eb73fe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3095eedd-ae83-4098-a8c3-d0c183eb73fe', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2'}, 'RetryAttempts': 0}}


In [110]:
exec_arn = "arn:aws:states:us-east-1:133240237239:execution:ce001005-a92a-44c2-abd6-a968d222f78e_152106-FR-20210526:ce001005-a92a-44c2-abd6-a968d222f78e_152106-FR-20210526-110334"

In [115]:
exec_arn.split(":")[-2]

'ce001005-a92a-44c2-abd6-a968d222f78e_152106-FR-20210526'